# Fast-Fourier-Transformation Example

## Machine-Learned Inverse FFT

Let's start with a sinusoidal curve: that's the input (`x`) that we want to learn, given its FFT (`y`).
The goal is to do an `InverseRealFFT` by gradient descent.

For this problem the input (`x`) is real (`float32`) and label (`y`), the FFT, is complex (`complex64`).

In [1]:
!*rm -f go.work && go work init && go work use . "${HOME}/Projects/gomlx"
%goworkfix

	- Added replace rule for module "github.com/gomlx/gomlx" to local directory "/home/janpf/Projects/gomlx".


In [2]:
import (
    "github.com/gomlx/gopjrt/dtypes"
    . "github.com/gomlx/gomlx/graph"
    . "github.com/gomlx/exceptions"
    "github.com/gomlx/gomlx/types/tensors"
    "github.com/gomlx/gomlx/types/xslices"
    mg "github.com/gomlx/gomlx/examples/notebook/gonb/margaid"
    "github.com/janpfeifer/gonb/gonbui"

    _ "github.com/gomlx/gomlx/backends/xla"
)

// manager always created at initialization.
var backend = backends.New()

const (
    NumPoints = 100
    Frequency = 2.0  // Number of curves in samples.
    RealDType = dtypes.Float32
    ComplexDType = dtypes.Complex64
)

// CalculateXY returns (x, y) of our problem, where y is a sinusoidal curve and x is its FFT.
func CalculateXY() (x, y *tensors.Tensor) {
    e := NewExec(backend, func (g *Graph) (x, y *Node) {
        x = Iota(g, shapes.Make(RealDType, 1, NumPoints), 1)
        x = MulScalar(x, 2.0*math.Pi*Frequency/float64(NumPoints))
        x = Sin(x)
        y = RealFFT(x)
        return
    })
    res := e.Call()
    return res[0], res[1]
}

func Plot(displayId string, width, height int, coordinates []*tensors.Tensor, names []string) {
    plts := mg.New(width, height)
    for ii, t := range coordinates {
        var values []float64
        switch t.DType() {
        case dtypes.Float64:
            values = tensors.CopyFlatData[float64](t)
        case dtypes.Float32:
            values32 := tensors.CopyFlatData[float32](t)
            values = xslices.Map(values32, func (v float32) float64 { return float64(v) })
        default:
            Panicf("only float32 and float64 tensor dtypes are accepted by Plot, got t.shape=%s", t.Shape())
        }
        var name string
        if len(names) > ii {
            name = names[ii]
        }
        plts.AddValues(name, "", values)
    }
    if displayId == "" {
        plts.Plot()
    } else {
        gonbui.UpdateHTML(displayId, plts.PlotToHTML())
    }
}

%%
x, y := CalculateXY()
fmt.Printf("x: shape=%s\n", x.Shape())
fmt.Printf("y: shape=%s\n", y.Shape())
Plot("", 1024, 320, []*tensors.Tensor{x}, nil)

x: shape=(Float32)[1 100]
y: shape=(Complex64)[1 51]


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,9.000000e+01 L2.625212e+01,1.008502e+02 L3.482424e+01,1.115293e+02 L4.339636e+01,1.218689e+02 L5.196848e+01,1.317058e+02 L6.054061e+01,1.408851e+02 L6.911273e+01,1.492618e+02 L7.768485e+01,1.567040e+02 L8.625697e+01,1.630942e+02 L9.482909e+01,1.683316e+02 L1.034012e+02,1.723338e+02 L1.119733e+02,1.750374e+02 L1.205455e+02,1.764000e+02 L1.291176e+02,1.764000e+02 L1.376897e+02,1.750374e+02 L1.462618e+02,1.723337e+02 L1.548339e+02,1.683316e+02 L1.634061e+02,1.630942e+02 L1.719782e+02,1.567040e+02 L1.805503e+02,1.492618e+02 L1.891224e+02,1.408851e+02 L1.976945e+02,1.317058e+02 L2.062667e+02,1.218688e+02 L2.148388e+02,1.115293e+02 L2.234109e+02,1.008502e+02 L2.319830e+02,9.000000e+01 L2.405552e+02,7.914979e+01 L2.491273e+02,6.847070e+01 L2.576994e+02,5.813114e+01 L2.662715e+02,4.829417e+01 L2.748436e+02,3.911494e+01 L2.834158e+02,3.073819e+01 L2.919879e+02,2.329603e+01 L3.005600e+02,1.690582e+01 L3.091321e+02,1.166837e+01 L3.177042e+02,7.666243e+00 L3.262764e+02,4.962576e+00 L3.348485e+02,3.600005e+00 L3.434206e+02,3.600005e+00 L3.519927e+02,4.962587e+00 L3.605648e+02,7.666258e+00 L3.691370e+02,1.166838e+01 L3.777091e+02,1.690585e+01 L3.862812e+02,2.329605e+01 L3.948533e+02,3.073821e+01 L4.034255e+02,3.911497e+01 L4.119976e+02,4.829422e+01 L4.205697e+02,5.813120e+01 L4.291418e+02,6.847076e+01 L4.377139e+02,7.914981e+01 L4.462861e+02,9.000002e+01 L4.548582e+02,1.008502e+02 L4.634303e+02,1.115293e+02 L4.720024e+02,1.218689e+02 L4.805745e+02,1.317058e+02 L4.891467e+02,1.408851e+02 L4.977188e+02,1.492618e+02 L5.062909e+02,1.567040e+02 L5.148630e+02,1.630942e+02 L5.234352e+02,1.683316e+02 L5.320073e+02,1.723338e+02 L5.405794e+02,1.750374e+02 L5.491515e+02,1.764000e+02 L5.577236e+02,1.764000e+02 L5.662958e+02,1.750374e+02 L5.748679e+02,1.723337e+02 L5.834400e+02,1.683316e+02 L5.920121e+02,1.630941e+02 L6.005842e+02,1.567039e+02 L6.091564e+02,1.492618e+02 L6.177285e+02,1.408850e+02 L6.263006e+02,1.317058e+02 L6.348727e+02,1.218688e+02 L6.434448e+02,1.115292e+02 L6.520170e+02,1.008501e+02 L6.605891e+02,9.000000e+01 L6.691612e+02,7.914979e+01 L6.777333e+02,6.847070e+01 L6.863055e+02,5.813114e+01 L6.948776e+02,4.829417e+01 L7.034497e+02,3.911493e+01 L7.120218e+02,3.073817e+01 L7.205939e+02,2.329602e+01 L7.291661e+02,1.690582e+01 L7.377382e+02,1.166836e+01 L7.463103e+02,7.666243e+00 L7.548824e+02,4.962576e+00 L7.634545e+02,3.600000e+00 L7.720267e+02,3.600010e+00 L7.805988e+02,4.962597e+00 L7.891709e+02,7.666274e+00 L7.977430e+02,1.166841e+01 L8.063152e+02,1.690588e+01 L8.148873e+02,2.329609e+01 L8.234594e+02,3.073825e+01 L8.320315e+02,3.911502e+01 L8.406036e+02,4.829427e+01 L8.491758e+02,5.813117e+01 L8.577479e+02,6.847074e+01 L8.663200e+02,7.914983e+01 "/> 0 10 20 30 40 50 60 70 80 90 Steps -0.000

### Train the model

If you run it, you'll see the plot of the "learnedX" adjusting towards "x", the original sinusoidal curve.

In [3]:
import (
    . "github.com/gomlx/gomlx/graph"
    "github.com/gomlx/gomlx/ml/context"
    "github.com/gomlx/gomlx/ml/data"
    "github.com/gomlx/gomlx/ml/train"
)

var (
	flagNumSteps     = flag.Int("steps", 1000, "Number of gradient descent steps to perform")
	flagLearningRate = flag.Float64("learning_rate", 0.1, "Initial learning rate.")
)

func TrainInverseRealFFT() {
    x, y := CalculateXY()
    ctx := context.New()
	ctx.SetParam(optimizers.LearningRateKey, *flagLearningRate)

    modelFn := func(ctx *context.Context, spec any, inputs []*Node) []*Node {
        g := inputs[0].Graph()
        learnedXVar := ctx.VariableWithShape("learnedX", x.Shape())
        predictedY := RealFFT(learnedXVar.ValueGraph(g))
        return []*Node{predictedY}
    }

    dataset, err := data.InMemoryFromData(backend, "dataset", []any{x}, []any{y})
    if err != nil {
        panic(err)
    }
    dataset.BatchSize(1, false).Infinite(true)

    opt := optimizers.Adam().Done()
    trainer := train.NewTrainer(
        backend, ctx, modelFn,
        losses.MeanAbsoluteError,
        opt,
        nil, nil) // trainMetrics, evalMetrics

	loop := train.NewLoop(trainer)
	commandline.AttachProgressBar(loop) // Attaches a progress bar to the loop.

    // Plot learnedX
    displayId := gonbui.UniqueID()
    gonbui.UpdateHTML(displayId, "")
    train.EveryNSteps(loop, 10, "plot", 0, func(loop *train.Loop, metrics []*tensors.Tensor) error {
        learnedXVar := ctx.InspectVariable(context.RootScope, "learnedX")
        learnedX := learnedXVar.Value()
        Plot(displayId, 1024, 320, []*tensors.Tensor{x, learnedX}, []string{"Truth", "Learned"})
        return nil
    })
    
	// Loop for given number of steps.
	_, err = loop.RunSteps(dataset, *flagNumSteps)
	if err != nil {
		panic(err)
	}

}

%% --steps=800 --learning_rate=0.01
TrainInverseRealFFT()

<path vector-effect="non-scaling-stroke" d="M1.768000e+01,8.999408e+01 L2.625212e+01,1.006324e+02 L3.482424e+01,1.114429e+02 L4.339636e+01,1.217591e+02 L5.196848e+01,1.317401e+02 L6.054061e+01,1.408296e+02 L6.911273e+01,1.490676e+02 L7.768485e+01,1.564052e+02 L8.625697e+01,1.628939e+02 L9.482909e+01,1.681826e+02 L1.034012e+02,1.720601e+02 L1.119733e+02,1.747557e+02 L1.205455e+02,1.763104e+02 L1.291176e+02,1.761916e+02 L1.376897e+02,1.749306e+02 L1.462618e+02,1.721935e+02 L1.548339e+02,1.682005e+02 L1.634061e+02,1.629267e+02 L1.719782e+02,1.565232e+02 L1.805503e+02,1.491594e+02 L1.891224e+02,1.407652e+02 L1.976945e+02,1.316201e+02 L2.062667e+02,1.220323e+02 L2.148388e+02,1.115560e+02 L2.234109e+02,1.010858e+02 L2.319830e+02,9.010221e+01 L2.405552e+02,7.914996e+01 L2.491273e+02,6.843484e+01 L2.576994e+02,5.819714e+01 L2.662715e+02,4.845352e+01 L2.748436e+02,3.906075e+01 L2.834158e+02,3.066705e+01 L2.919879e+02,2.356296e+01 L3.005600e+02,1.698352e+01 L3.091321e+02,1.179975e+01 L3.177042e+02,7.549653e+00 L3.262764e+02,5.289318e+00 L3.348485e+02,3.770771e+00 L3.434206e+02,3.600000e+00 L3.519927e+02,5.283869e+00 L3.605648e+02,7.768969e+00 L3.691370e+02,1.186453e+01 L3.777091e+02,1.725516e+01 L3.862812e+02,2.343759e+01 L3.948533e+02,3.095035e+01 L4.034255e+02,3.917177e+01 L4.119976e+02,4.847085e+01 L4.205697e+02,5.826066e+01 L4.291418e+02,6.847814e+01 L4.377139e+02,7.914791e+01 L4.462861e+02,8.992999e+01 L4.548582e+02,1.006934e+02 L4.634303e+02,1.113845e+02 L4.720024e+02,1.217935e+02 L4.805745e+02,1.315418e+02 L4.891467e+02,1.405904e+02 L4.977188e+02,1.489791e+02 L5.062909e+02,1.565298e+02 L5.148630e+02,1.628629e+02 L5.234352e+02,1.681056e+02 L5.320073e+02,1.719722e+02 L5.405794e+02,1.747173e+02 L5.491515e+02,1.764000e+02 L5.577236e+02,1.761338e+02 L5.662958e+02,1.748204e+02 L5.748679e+02,1.722865e+02 L5.834400e+02,1.683800e+02 L5.920121e+02,1.630900e+02 L6.005842e+02,1.563718e+02 L6.091564e+02,1.490798e+02 L6.177285e+02,1.407051e+02 L6.263006e+02,1.315206e+02 L6.348727e+02,1.219027e+02 L6.434448e+02,1.114152e+02 L6.520170e+02,1.010800e+02 L6.605891e+02,8.987170e+01 L6.691612e+02,7.927377e+01 L6.777333e+02,6.838987e+01 L6.863055e+02,5.808632e+01 L6.948776e+02,4.848491e+01 L7.034497e+02,3.898492e+01 L7.120218e+02,3.051974e+01 L7.205939e+02,2.360547e+01 L7.291661e+02,1.693577e+01 L7.377382e+02,1.184585e+01 L7.463103e+02,7.797864e+00 L7.548824e+02,5.130147e+00 L7.634545e+02,3.700341e+00 L7.720267e+02,3.734804e+00 L7.805988e+02,4.980813e+00 L7.891709e+02,7.677853e+00 L7.977430e+02,1.173978e+01 L8.063152e+02,1.713445e+01 L8.148873e+02,2.342989e+01 L8.234594e+02,3.067704e+01 L8.320315e+02,3.932203e+01 L8.406036e+02,4.844448e+01 L8.491758e+02,5.809368e+01 L8.577479e+02,6.860087e+01 L8.663200e+02,7.930597e+01 "/> <path d="M1.768000e+01,8.998663e+01 L2.625212e+01,1.008172e+02 L3.482424e+01,1.114770e+02 L4.339636e+01,1.217978e+02 L5.196848e+01,1.316170e+02 L6.054061e+01,1.407796e+02 L6.911273e+01,1.491413e+02 L7.768485e+01,1.565700e+02 L8.625697e+01,1.629486e+02 L9.482909e+01,1.681766e+02 L1.034012e+02,1.721715e+02 L1.119733e+02,1.748702e+02 L1.205455e+02,1.762304e+02 L1.291176e+02,1.762304e+02 L1.376897e+02,1.748702e+02 L1.462618e+02,1.721715e+02 L1.548339e+02,1.681766e+02 L1.634061e+02,1.629486e+02 L1.719782e+02,1.565699e+02 L1.805503e+02,1.491413e+02 L1.891224e+02,1.407796e+02 L1.976945e+02,1.316170e+02 L2.062667e+02,1.217978e+02 L2.148388e+02,1.114770e+02 L2.234109e+02,1.008172e+02 L2.319830e+02,8.998662e+01 L2.405552e+02,7.915604e+01 L2.491273e+02,6.849626e+01 L2.576994e+02,5.817540e+01 L2.662715e+02,4.835622e+01 L2.748436e+02,3.919360e+01 L2.834158e+02,3.083199e+01 L2.919879e+02,2.340329e+01 L3.005600e+02,1.702465e+01 L3.091321e+02,1.179666e+01 L3.177042e+02,7.801779e+00 L3.262764e+02,5.103002e+00 L3.348485e+02,3.742896e+00 L3.434206e+02,3.742896e+00 L3.519927e+02,5.103013e+00 L3.605648e+02,7.801794e+00 L3.691370e+02,1.179668e+01 L3.777091e+02,1.702467e+01 L3.862812e+02,2.340332e+01 L3.948533e+02,3.083202e+01 L4.034255e+02,3.919363e+01 L

Training (800 steps):  100% [========================================] (1817 steps/s) [step=799] [loss+=0.012] [~loss+=0.022] [~loss=0.022]        
